# U-net model

![](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)

In [1]:
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Cropping2D, Dropout, ZeroPadding2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from utils import get_resized_train_data
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm

IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_DEPTH = 3
DROPOUT_RATE = 0.2

In [3]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [8]:
inps = Input((IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH))
# pad = ZeroPadding2D(((64, 64), (64, 64)))(inps)
conv1 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(inps)
conv1 = Dropout(.1)(conv1)
conv1 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
pool = MaxPooling2D(pool_size=(2,2))(conv1)
conv2 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(pool)
conv2 = Dropout(.1)(conv2)
conv2 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
conv3 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
conv3 = Dropout(.2)(conv3)
conv3 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
conv4 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
conv4 = Dropout(.2)(conv4)
conv4 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
conv5 = Conv2D(1024, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
conv5 = Dropout(.2)(conv5)
conv5 = Conv2D(1024, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

# start deconvoluting
deconv1 = Conv2DTranspose(512, (2,2), strides=(2,2), padding='same')(conv5)
# deconv1 = Conv2DTranspose(512, (2,2), padding='same')(conv5)
# cropped_conv4 = Cropping2D(cropping=((4, 4), (4, 4)))(conv4)
# deconv1 = concatenate([deconv1, cropped_conv4])
deconv1 = concatenate([deconv1, conv4])
conv6 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(deconv1)
conv6 = Dropout(.2)(conv6)
conv6 = Conv2D(512, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
deconv2 = Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(conv6)
# cropped_conv3 = Cropping2D(cropping=((16, 17), (16, 17)))(conv3)
# deconv2 = concatenate([deconv2, cropped_conv3])
deconv2 = concatenate([deconv2, conv3])
conv7 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(deconv2)
conv7 = Dropout(.2)(conv7)
conv7 = Conv2D(256, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
deconv3 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(conv7)
# cropped_conv2 = Cropping2D(cropping=((41, 41), (41, 41)))(conv2)
# deconv3 = concatenate([deconv3, cropped_conv2])
deconv3 = concatenate([deconv3, conv2])
conv8 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(deconv3)
conv8 = Dropout(.2)(conv8)
conv8 = Conv2D(128, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
deconv4 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(conv8)
# cropped_conv1 = Cropping2D(cropping=((90, 90), (90, 90)))(conv1)
# deconv4 = concatenate([deconv4, cropped_conv1])
deconv4 = concatenate([deconv4, conv1])
conv9 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(deconv4)
conv9 = Dropout(.2)(conv9)
conv9 = Conv2D(64, (3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
outs = Conv2D(1, (1,1), activation='sigmoid')(conv9)

model = Model(inputs=[inps], outputs=[outs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_80 (Conv2D)              (None, 256, 256, 64) 1792        input_6[0][0]                    
__________________________________________________________________________________________________
dropout_39 (Dropout)            (None, 256, 256, 64) 0           conv2d_80[0][0]                  
__________________________________________________________________________________________________
conv2d_81 (Conv2D)              (None, 256, 256, 64) 36928       dropout_39[0][0]                 
__________________________________________________________________________________________________
max_poolin

In [ ]:
import utils
from importlib import reload
reload(utils)
X_train, y_train = utils.get_resized_train_data()

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('unet-model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])# Define IoU metric

Train on 603 samples, validate on 67 samples
Epoch 1/50
